In [1]:
!pip install xgboost==1.4.2

     |████████████████████████████████| 166.7 MB 18 kB/s 
  Attempting uninstall: xgboost
    Found existing installation: xgboost 0.90
    Uninstalling xgboost-0.90:
      Successfully uninstalled xgboost-0.90


In [35]:
import random
import sys
import time
import os
import glob
import datetime
import pytz
import argparse

import numpy as np
import pandas as pd
import pickle
import joblib

import scipy
import sklearn
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report
from scipy.stats import skew, kurtosis
#from Vibration_Feature_Extractor import Extract_Time_Features, Extract_Freq_Features

In [36]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [37]:
class Extract_Time_Features:    
    def __init__(self, rawTimeData):
        self._TimeFeatures = []
        self._rawTimeData = rawTimeData

    def AbsMax(self):
        self._absmax = np.abs(self._rawTimeData).max(axis=1)
        return self._absmax
    
    def AbsMean(self):
        self._absmean = np.abs(self._rawTimeData).mean(axis=1)
        return self._absmean
    
    def P2P(self):
        self._peak2peak = np.max(self._rawTimeData,axis=1) - np.min(self._rawTimeData,axis=1)
        return self._peak2peak 
    
    def Skewness(self):
        self._skewness = skew(self._rawTimeData, axis=1, nan_policy='raise')
        return self._skewness
    
    def Kurtosis(self):
        self._kurtosis = kurtosis(self._rawTimeData, axis=1, fisher=False)
        return self._kurtosis

    def RMS(self):
        self._rms = np.sqrt(np.sum(self._rawTimeData**2, axis=1) / self._rawTimeData.shape[1])
        return self._rms
    
    def CrestFactor(self):
        self._cresfactor = self.P2P() / self.RMS()
        return self._cresfactor
    
    def ShapeFactor(self):
        self._shapefactor = self.RMS() / self.AbsMean()
        return self._shapefactor
    
    def ImpulseFactor(self):
        self._impulsefactor = self.AbsMax() / self.AbsMean()
        return self._impulsefactor

    def Features(self):
        # Time-domain Features #
        self._TimeFeatures.append(self.AbsMax())        # Feature 1: Absolute Maximum 
        self._TimeFeatures.append(self.AbsMean())       # Feature 2: Absolute Mean
        self._TimeFeatures.append(self.P2P())           # Feature 3: Peak-to-Peak
        self._TimeFeatures.append(self.RMS())           # Feature 4: Root-mean-square
        self._TimeFeatures.append(self.Skewness())      # Feature 5: Skewness
        self._TimeFeatures.append(self.Kurtosis())      # Feature 6: Kurtosis
        self._TimeFeatures.append(self.CrestFactor())   # Feature 7: Crest Factor
        self._TimeFeatures.append(self.ShapeFactor())   # Feature 8: Shape Factor
        self._TimeFeatures.append(self.ImpulseFactor()) # Feature 9: Impulse Factor
                
        return np.asarray(self._TimeFeatures)

In [39]:
def framework():
    print('########## 진동 유효 검증 시작 #############')
    print('시작시간(UCT+09:00)',datetime.datetime.now(pytz.timezone('Asia/Seoul')).strftime("%Y-%m-%d %H:%M:%S"))
    print('_Python.version:', sys.version)
    print('_Scipy.version:',scipy.__version__)
    print('_XGBoost.version:',xgb.__version__)
    print('_Sklearn.version:',sklearn.__version__)

In [40]:
def validation(filename):
        
    try:
        vib = pd.read_csv(filename, header=None, skiprows=9)
        vib = np.asarray(vib)[:,1:-1].transpose()

        meta = pd.read_csv(filename, header=None, skiprows=4, nrows=1)
        rpm = int(meta[2])

        fs = pd.read_csv(filename, header=None, skiprows=6, nrows=1)
        Fs = int(fs[1])

        TimeFeatureExtractor = Extract_Time_Features(vib)
        features_time = TimeFeatureExtractor.Features()

        FreqFeatureExtractor = Extract_Freq_Features(vib, rpm, Fs)
        features_freq = FreqFeatureExtractor.Features()

        features = np.concatenate((features_time, features_freq)).T
        data = pd.DataFrame(features)

        data.columns = ['AbsMax', 'AbsMean', 'P2P', 'RMS', 'Skewness', 'Kurtosis', 'Crest', 
                        'Shape', 'Impulse','1x', '2x', '3x', '4x', '1xB', '2xB', '3xB', '4xB']

        data['WATT'] = meta[3]
        y = pd.read_csv(filename, header=None, skiprows=3, nrows=1)
        y = y[1]
        #print("done?")

    except (NameError, IndexError, TypeError, pd.errors.EmptyDataError):
        print('Source Data Error')
        
    else:
        df = data[['WATT', 'AbsMax', 'AbsMean', 'P2P', 'RMS', 'Skewness', 'Kurtosis', 'Crest', 
                        'Shape', 'Impulse','1x', '2x', '3x', '4x', '1xB', '2xB', '3xB', '4xB']]

        norm = np.load('/content/drive/MyDrive/SFEP/vib_dataset/vibration_norm.npy')#norm
        X = (df - norm[0]) / norm[1]

        xgb_joblib = joblib.load('/content/drive/MyDrive/SFEP/vib_dataset/vibration_xgb.pkl') 
        y_pred = xgb_joblib.predict(X)
        name = os.path.basename(filename)
        print("Filename:%s, Prediction:%s, Y_Target:%d" % (name, y_pred[0], y))
        
    return y_pred[0], y[0]

In [41]:
datapath = glob.glob("/content/drive/MyDrive/SFEP/vib_dataset/" + '**/**/**/**/*.csv', recursive=True)

In [42]:
len(datapath)

46800

In [43]:
start = time.time()

# parser = argparse.ArgumentParser()
# parser.add_argument('--path', default='vibration', type=str)
# args = parser.parse_args()
# datapath = glob.glob(args.path + '**/**/*.csv', recursive=True)

framework()

#random = random.sample(datapath, 100)

y_hat = []
y_target = []
for file in datapath:
    y_pred, y = validation(file)
    y_hat.append(y_pred)
    y_target.append(y)
    
target_names = ['0', '1', '2', '3', '4']
print('########## 진동데이터 유효검증 결과 ##########')
print(f"검증데이터수: {len(y_hat)} 개")

accuracy = accuracy_score(y_target, y_hat)
print('예측 정확도: {0:.4f}'.format(accuracy))
print(classification_report(y_target, y_hat, target_names=target_names, digits=4)) 
print('종료시간(UCT+09:00)',datetime.datetime.now(pytz.timezone('Asia/Seoul')).strftime("%Y-%m-%d %H:%M:%S"))
end = time.time()
elapsed_time = end-start
print(f"검증소요시간：{elapsed_time}초")

y_prediction = pd.DataFrame(y_hat)
y_prediction.to_csv('y_predict_vibration.csv', index=False)

########## 진동 유효 검증 시작 #############
시작시간(UCT+09:00) 2021-08-09 21:03:06
_Python.version: 3.7.11 (default, Jul  3 2021, 18:01:19) 
[GCC 7.5.0]
_Scipy.version: 1.4.1
_XGBoost.version: 1.4.2
_Sklearn.version: 0.22.2.post1
Source Data Error


UnboundLocalError: ignored